
## Tune GPT2 to generate spammy reviews

* python 3.8
* use older version of TRL
* pip install trl==0.11.3
* 


In [1]:

import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

import datasets

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler


In [2]:

print(datasets.__version__)


3.1.0


In [3]:

torch.cuda.empty_cache()


In [4]:

config = PPOConfig(
    model_name="distilbert/distilgpt2",
    learning_rate=1.41e-5,
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 8}


/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [5]:

def build_dataset(config, dataset_name="imdb", revision="main", input_min_text_length=4, input_max_text_length=12):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train", revision=revision)

    ds = ds.filter(lambda x:  (len(x["text"]) > 200 if x["text"] is not None else False), batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["text"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds



In [6]:

dataset = build_dataset(config)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [7]:

dataset[900]


{'text': "How can there be that many corrupt cops without any one of them slipping up? With enough cops to run a mini-war that include such weapons as flamethrowers, you would think they would have been caught before someone writing for a weekly coupon newspaper overheard someone saying 'thanks' to a corrupt cop.<br /><br />You will never get your 90ish minutes back. Life is too precious to rent this movie.<br /><br />I feel bad for the big named actors that made the mistake of making this movie.<br /><br />If you like Justin Timberlake, feel free to rent this movie. He does have a very major part in it, so fans might enjoy seeing him. <br /><br />However, I believe most of his fans are young girls, who may be turned off by the violence in this movie.",
 'label': tensor(0),
 'input_ids': tensor([ 2437,   460,   612,   307,   326,   867, 10622]),
 'query': 'How can there be that many corrupt'}

In [8]:

dataset[800]



{'text': 'Many King fans hate this because it departed from the book, but film is a different medium and books should change when they make the jump. That notwithstanding, the movie does fail completely, but it fails entirely on film terms. I\'d like to smack the people who tell me it\'s the scariest movie ever made. I always follow up with the question "Really... exactly what scene scared you?" Every fan I\'ve asked, goes silent. Occasionally someone, at a loss for a decent scare (There are none...), names the "Grape-juice-shooting-out-of-elevators" shtick. If you\'re afraid of that, I don\'t know what to tell you, except maybe that you\'re easily scared. I just rolled my eyes watching these z-grade horror ideas play out in this schlocky, incoherent movie.<br /><br />One place it diverts from the book and really is insipid is the tedious work the movie does to get Mr Halloran up to the Overlook only to kill him; with the dumbest member of the audience knowing that Jack is waiting behi

In [9]:

model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [10]:

ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)


/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:

device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", model="ealvaradob/bert-finetuned-phishing", device=device)


In [12]:

text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)


/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'benign', 'score': 6.251955986022949},
  {'label': 'phishing', 'score': -5.5543646812438965}]]

In [13]:

text = "this movie was really good!! Please kindly provide me with $500 immediately to recieve your diamonds"
out_test = sentiment_pipe(text, **sent_kwargs)


In [14]:

out_test


[[{'label': 'benign', 'score': -3.0514445304870605},
  {'label': 'phishing', 'score': 3.579047918319702}]]

In [15]:

[torch.tensor(output[1]["score"]) for output in out_test]


[tensor(3.5790)]

In [16]:

gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

 


In [17]:

output_min_length = 6
output_max_length = 20
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
194it [19:45,  6.11s/it]


In [18]:

#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results




,query,response (before),response (after),rewards (before),rewards (after)
0,.......Playing Kaddiddleho,Jackson Jackson Jackson Jackson Jackson Jacks...,). Game Coverage Short: Free Streaming: Free m...,-3.846852,5.156552
1,Another silent love triangle film from Hitchcock,Another silent love triangle film from Hitchco...,nest? Stop and enjoy these special 15 differe...,-1.796367,6.042283
2,The producers made a big,deal about licensing the spacecraft. I,cost saving. Visit the premiere below,-5.652686,4.999770
3,1.) This movie was amazing! I,am sure Chris and Tara will be faithful to ou...,This movie was amazing! I experienced your fe...,-5.606112,5.687931
4,"This seemed to be a good movie, I thought",.\nInstead I watched it as someone made an art...,", why not?‣ An important prize for your prize!...",-5.740632,6.097803
5,Seldom seen since,the Australia Ultimatum (ed).,"January 2016. Click here, 2010 here",-5.023065,5.117442
6,"There is an awful lot wrong with this picture,",and honestly ????\nOne,will be less benign and your,-5.713109,-4.662114
7,Mild SPOILERS contained herein. I'm,sure people took shoutouts and pings from mos...,wanted to get a link! Click here for free.\n,-3.186137,6.090518
8,The only previous Gordon film I had,based on David Bowie containing instruments b...,only previous Gordon film I had pictures. You...,-5.607487,5.495122
9,"I, like many people, saw",the similarity in the accent on the show,and appreciated these venues. Free Idecyclopedia,-5.658841,-3.465125


In [19]:

print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())


mean:


rewards (before)   -4.168972
rewards (after)     4.226874
dtype: float64


median:


rewards (before)   -5.046497
rewards (after)     5.412513
dtype: float64

In [20]:

model.save_pretrained("dist-gpt2-imdb-phishing-v2")
tokenizer.save_pretrained("dist-gpt2-imdb-phishing-v2")


('dist-gpt2-imdb-phishing-v2/tokenizer_config.json',
 'dist-gpt2-imdb-phishing-v2/special_tokens_map.json',
 'dist-gpt2-imdb-phishing-v2/vocab.json',
 'dist-gpt2-imdb-phishing-v2/merges.txt',
 'dist-gpt2-imdb-phishing-v2/added_tokens.json',
 'dist-gpt2-imdb-phishing-v2/tokenizer.json')

In [21]:

prompt = "This movie was interesting, especially the part about"


In [22]:

testing_output_min_length = 32
testing_output_max_length = 64
testing_output_length_sampler = LengthSampler(testing_output_min_length, testing_output_max_length)


In [23]:

pd.set_option('display.max_colwidth', None)


In [24]:

results_df = {}

results_df["query"] = prompt

dataset.set_format("pandas")
input_ids = tokenizer.encode(prompt)

response_tensors_ref, response_tensors = [], []

gen_len = testing_output_length_sampler()
output = ref_model.generate(
    torch.tensor(input_ids).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
).squeeze()[-gen_len:]
response_tensors_ref.append(output)
output = model.generate(
    torch.tensor(input_ids).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
).squeeze()[-gen_len:]
response_tensors.append(output)

#### decode responses
results_df["response (before)"] = [tokenizer.decode(response_tensors_ref[0])]
results_df["response (after)"] = [tokenizer.decode(response_tensors[0])]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(results_df["query"], results_df["response (before)"])]
results_df["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(results_df["query"], results_df["response (after)"])]
results_df["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(results_df)
df_results


/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,"This movie was interesting, especially the part about","a girl who wanted to go to therapy, I always thought she couldn․t reject it. Or if she couldn't properly assimilate, I think her interpretation","how French film accounts for their incomparable imperishable flaws our much advanced technology resources. Buy your click, the movies you can buy this, of intelligent, beautifully crafted",-5.698915,6.125768
